In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
pwd

'/kaggle/working'

In [10]:
!wget -O /kaggle/working/Vision.zip "https://www.dropbox.com/s/mamwc19rgy8wiv5/Vision.zip?dl=1"

--2023-04-07 06:49:49--  https://www.dropbox.com/s/mamwc19rgy8wiv5/Vision.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/mamwc19rgy8wiv5/Vision.zip [following]
--2023-04-07 06:49:50--  https://www.dropbox.com/s/dl/mamwc19rgy8wiv5/Vision.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc9499c3379470bbd186e0faf40.dl.dropboxusercontent.com/cd/0/get/B5tKisE1HS58KXQF6UTLxPgk1tWLgw9odDe4VIeUJW9VxRfLL-I96do8Ep7G5yW4ob-KP8zizBi3h0qD53nSjSSGZMIiLf3PzBeGRwXk0gbJ-MTYZwVcQCSml_6ez49YeCjiUTXPLWVS9CH_B41tBuR6zgY1O23Cxth0vKuyfTfyfamcu8-cfIfRiLRnehzyf7s/file?dl=1# [following]
--2023-04-07 06:49:50--  https://ucc9499c3379470bbd186e0faf40.dl.dropboxusercontent.com/cd/0/get/B5tKisE1HS58KXQF6UTLxPgk1tWLgw9odDe4VIeUJW9VxRfLL-I96do8Ep7G5yW

In [11]:
!unzip Vision.zip -d "/kaggle/working/"

Archive:  Vision.zip
replace /kaggle/working/Vision/Scenario5/challenge_dataset/scenario5_challenge.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [14]:
import os
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm

# define root directory
ROOT = os.getcwd()
print(ROOT)

dataset_dir = os.path.join(ROOT,"Vision")
scenario_dir = os.path.join(dataset_dir,"Scenario5")
development_dir = os.path.join(scenario_dir, "development_dataset")
challenge_dir = os.path.join(scenario_dir, "challenge_dataset")

/kaggle/working


In [15]:
path = '/kaggle/working/Vision'
checkpoint = 'checkpoint'

for p in os.listdir(path) :
    os.mkdir(os.path.join(path,p,checkpoint))

FileExistsError: [Errno 17] File exists: '/kaggle/working/Vision/Scenario6/checkpoint'

In [16]:
import random

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available() :
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [17]:
# load data

import torchvision
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms.functional as FT

def transform(image, flag) :
    mean = [0.485, 0.456, 0.406]
    std =  [0.229, 0.224, 0.225]

    if flag == 2 :      #test
        new_image = FT.resize(image,224)
        new_image = FT.to_tensor(new_image)
        new_image = FT.normalize(new_image, mean=mean, std=std)
    else :              #train, val
        new_image = FT.resize(image,224)
        new_image = FT.to_tensor(new_image)
        new_image = FT.normalize(new_image, mean=mean, std=std)

    return new_image

class DeepSense6G(Dataset):
    def __init__(self, data_folder, split, flag):
        self.split = split
        assert self.split in {'Scenario5','Scenario6','Scenario7','Scenario8','Scenario9'}

        self.data_folder = data_folder
        self.scenario_path = os.path.join(self.data_folder, split)
        self.development_path = os.path.join(self.scenario_path, 'development_dataset')
        self.challenge_path = os.path.join(self.scenario_path, 'challenge_dataset')

        self.flag = flag

        self.split = self.split.lower()
        if flag == 0 :      #train
            self.dataframe_path = os.path.join(self.development_path, self.split + '_dev_train.csv')
        elif flag == 1 :    #validation
            self.dataframe_path = os.path.join(self.development_path, self.split + '_dev_val.csv')
        elif flag == 2 :              #test
            self.dataframe_path = os.path.join(self.development_path, self.split + '_dev_test.csv')

        self.dataframe = pd.read_csv(self.dataframe_path)

        self.Images_path = self.dataframe['unit1_rgb_1'].values
        self.Beam_index = self.dataframe['beam_index_1'].values
        self.image_index = self.dataframe['index'].values
        self.Beam_pwr_path = self.dataframe['unit1_pwr_1'].values

    def __getitem__(self, i):

        self.img_rel_path = self.Images_path[i]

        if self.flag == 2 :     #test
            self.img_abs_path = os.path.join(self.development_path, self.img_rel_path)
        else :                  #train, validation
            self.img_abs_path = os.path.join(self.development_path, self.img_rel_path)
        
        image = Image.open(self.img_abs_path, mode='r')
        image = image.convert('RGB')

        beam_index = self.Beam_index[i]

        image = transform(image, self.flag)

        return image, beam_index

    def __len__(self):
        return len(self.Images_path)

    # def _collate_fn(self, batch_size) :

    #     images = list()
    #     beam_indexes = list()

    #     for b in batch_size :
    #         images.append(b[0])
    #         beam_indexes.append(b[1])

    #     images = torch.stack(images, dim=0)

    #     return images, beam_indexes

In [18]:
# device initialize

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(DEVICE)

cuda


## wideResnet101

In [19]:
#wide_resnet101_2

wide_resnet101 = torch.hub.load('pytorch/vision:v0.10.0', 'wide_resnet101_2', pretrained=True)
num_ftrs = wide_resnet101.fc.in_features
wide_resnet101.fc = torch.nn.Linear(num_ftrs, 64)

model = wide_resnet101.to(DEVICE)
print(model)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Wide_ResNet101_2_Weights.IMAGENET1K_V1`. You can also use `weights=Wide_ResNet101_2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), strid

In [20]:
#training
import time

batch_size = 32
scenario = 'Scenario5'

train_dataset = DeepSense6G(dataset_dir, split=scenario, flag=0)
val_dataset = DeepSense6G(dataset_dir, split=scenario, flag=1)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False) 

epochs = 30
best_loss = 100

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[12,20], gamma=0.2)
loss = torch.nn.CrossEntropyLoss().cuda()

train_acc_list = []
val_acc_list = []
lrs = []

for epoch in range(epochs) :
    start = time.time()
    print(f'Epoch {epoch}/{epochs}')

    #train
    train_loss = 0
    correct = 0

    model.train()
    for img, bi in tqdm(train_loader) :

        optimizer.zero_grad()
        y_pred = model(img.cuda())
        cost = loss(y_pred, bi.cuda())

        cost.backward()
        optimizer.step()
        lrs.append(optimizer.param_groups[0]["lr"])
        scheduler.step()
        
        train_loss += cost.item()

        pred = y_pred.data.max(1, keepdim=True)[1]
        correct += pred.cpu().eq(bi.data.view_as(pred)).sum()

    train_loss /= len(train_loader)
    train_acc = correct / len(train_loader.dataset)
    train_acc_list.append(train_acc)

    #validate
    val_loss = 0
    val_correct = 0

    with torch.no_grad() :
        model.eval()
        for img, bi in val_loader :
            y_pred = model(img.cuda())
            cost = loss(y_pred, bi.cuda())
            val_loss += cost.item()

            pred = y_pred.data.max(1, keepdim=True)[1]
            val_correct += pred.cpu().eq(bi.data.view_as(pred)).cpu().sum()
        
        val_loss /= len(val_loader)
        val_acc = val_correct / len(val_loader.dataset)
        val_acc_list.append(val_acc)

        if val_loss < best_loss :
            torch.save({
                'epoch' : epoch,
                'model' : model,
                'model_state_dict' : model.state_dict(),
                'optimizer_state_dict' : optimizer.state_dict(),
                'loss' : cost.item,
            }, os.path.join(dataset_dir, scenario + '/checkpoint/bestCheckpoint.pth'))

            print(f'Epoch {epoch:05d}: val_loss improved from {best_loss:.5f} to {val_loss:.5f}, saving model to bestCheckPoint.pth')
            best_loss = val_loss
        else :
            print(f'Epoch {epoch:05d}: val_loss did not improve')
    print(f'{int(time.time() - start)}s - loss: {train_loss: .5f} - acc: {train_acc:.5f} - val_loss: {val_loss:.5f} - val_acc: {val_acc:.5f}')

Epoch 0/30


100%|██████████| 50/50 [01:39<00:00,  1.98s/it]


Epoch 00000: val_loss improved from 100.00000 to 3.18667, saving model to bestCheckPoint.pth
121s - loss:  3.51576 - acc: 0.23801 - val_loss: 3.18667 - val_acc: 0.34586
Epoch 1/30


100%|██████████| 50/50 [01:33<00:00,  1.87s/it]


Epoch 00001: val_loss improved from 3.18667 to 2.97053, saving model to bestCheckPoint.pth
116s - loss:  3.06531 - acc: 0.34533 - val_loss: 2.97053 - val_acc: 0.37343
Epoch 2/30


...

Epoch 00027: val_loss improved from 1.44495 to 1.43085, saving model to bestCheckPoint.pth
115s - loss:  0.44310 - acc: 0.93687 - val_loss: 1.43085 - val_acc: 0.57143
Epoch 28/30


100%|██████████| 50/50 [01:33<00:00,  1.87s/it]


Epoch 00028: val_loss did not improve
112s - loss:  0.41336 - acc: 0.94318 - val_loss: 1.43860 - val_acc: 0.57143
Epoch 29/30


100%|██████████| 50/50 [01:33<00:00,  1.87s/it]


Epoch 00029: val_loss did not improve
112s - loss:  0.38705 - acc: 0.94760 - val_loss: 1.43330 - val_acc: 0.55388


In [21]:
## 저장되어있는 best checkpoint load

best_model = torch.load(os.path.join(dataset_dir, scenario + '/checkpoint/bestCheckpoint.pth'))['model'] 
best_model.load_state_dict(torch.load(os.path.join(dataset_dir, scenario + '/checkpoint/bestCheckpoint.pth'))['model_state_dict'])  

<All keys matched successfully>

In [22]:
#test 
import math

batch_size = 1

test_dataset = DeepSense6G(dataset_dir, split=scenario, flag=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

correct = 0
total = 0
total_results = []
gt = []

def top_k (gts, preds, k) :
    correct = 0

    for i in range(len(preds)) :
        gt = gts[i]
        pred = preds[i]
        correct += int(gt.item() in pred[:k])
    
    print(scenario, 'top', k, 'accuracy', 100 * correct / len(preds))

def mse (gts, preds) :
    mse = 0

    for i in range(len(preds)) :
        gt = gts[i].item()
        pred = preds[i][0]
        mse += (gt-pred)**2
    
    print(scenario, 'mse', mse/len(preds))

with torch.no_grad() :
    model.eval()
    for img, bi in test_loader :
        pred = best_model(img.cuda())
        sorted_result_idx = np.flip(np.array(np.argsort(pred.cpu())[0]))
        top3 = sorted_result_idx[:3]
        total_results.append(top3)
        gt.append(bi)

top_k(gt, total_results, 1)
top_k(gt, total_results, 2)
top_k(gt, total_results, 3)
mse(gt, total_results)

Scenario5 top 1 accuracy 63.722397476340696
Scenario5 top 2 accuracy 83.91167192429022
Scenario5 top 3 accuracy 94.00630914826499
Scenario5 mse 0.9779179810725552
